In [7]:
from matminer.featurizers.composition import ElementFraction
from pymatgen.symmetry.groups import SpaceGroup
from pymatgen.core import Composition
import pandas as pd

---- 
Initial Tests for getting compositional descriptors and Spacegroup Numbers.

In [11]:
df = pd.read_csv('./Data/1_MatDX/MatDX_EF.csv')

structure_1 = df.iloc[0]
structure_1

formula                                                        Ni4Ta6
space_group                                                      R-3c
structure           [{'data': {'a': [-6.910143e-10, 0, 0], 'b': [3...
id                                PN_MCKVEQNTJFRUHOURCR2HCEB35KDFRETL
formation_energy    {'reference': {'Ni': 'https://nomad-lab.eu/pro...
Name: 0, dtype: object

In [3]:
def concatenate_lists(*lists):
    concatenated_list = [item for lst in lists for item in lst]
    return concatenated_list

In [6]:
formula = "Al3Ga4"
symbol = 'R-3c'

featurizer = ElementFraction()

sg = [SpaceGroup(symbol).int_number]
vol = [30]
composition = Composition(formula)

descriptor = featurizer.featurize(composition)


print("The new vector is", concatenate_lists(descriptor, sg, vol))
print("Descriptor:", descriptor)
print("Space group:", sg)
print('Volume:', vol)

The new vector is [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.42857142857142855, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5714285714285714, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 167, 30]
Descriptor: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.42857142857142855, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5714285714285714, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Space group: [167]
Volume: [30]
